## Install Libs

In [3]:
!pip3 install keras
!pip3 install ann_visualizer
!pip3 install graphviz
!pip3 install tensorflow

     |████████████████████████████████| 195.7 MB 83 kB/s s eta 0:00:0114     |██████████▍                     | 63.4 MB 4.9 MB/s eta 0:00:28     |█████████████▍                  | 81.6 MB 30.2 MB/s eta 0:00:04     |██████████████▍                 | 87.9 MB 30.2 MB/s eta 0:00:04     |██████████████████████▉         | 139.5 MB 90.8 MB/s eta 0:00:01     |███████████████████████▎        | 142.6 MB 90.8 MB/s eta 0:00:01██████████████▏       | 147.6 MB 5.7 MB/s eta 0:00:09     |████████████████████████▎       | 148.6 MB 5.7 MB/s eta 0:00:09     |█████████████████████████▋      | 156.3 MB 5.7 MB/s eta 0:00:07
     |████████████████████████████████| 57 kB 9.1 MB/s  eta 0:00:01
     |████████████████████████████████| 2.9 MB 20.6 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 16.6 MB/s eta 0:00:01
     |████████████████████████████████| 15.6 MB 18.1 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.9 MB/s  eta 0:00:01
     |████████████████████████████████| 6.0 MB

  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.0
    Uninstalling typing-extensions-3.10.0.0:
      Successfully uninstalled typing-extensions-3.10.0.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Attempting uninstall: h5py
    Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.11.2
    Uninstalling wrapt-1.11.2:
      Successfully uninstalled wrapt-1.11.2
  Attempting uninstall: wheel
    Found existing installation: wheel 0.34.2
    Uninstalling wheel-0.34.2:
      Successfully uninstalled wheel-0.34.2


## Import Libs

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from ann_visualizer.visualize import ann_viz
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Variables

In [5]:
_dir = "."
file_name = 'German_Dataset.csv'

In [6]:
_dict = {}

funcao_perda = "binary_crossentropy" #@param ["binary_crossentropy", "mean_squared_error"] {type: "string", allow-input: false}
funcao_ativacao_cam_oculta = "relu" #@param ["softmax", "elu", "selu", "softplus", "softsign", "relu", "tanh", "sigmoid", "hard_sigmoid", "exponential", "linear"] {type: "string", allow-input: false}
funcao_ativacao_cam_saida = "sigmoid" #@param ["softmax", "elu", "selu", "softplus", "softsign", "relu", "tanh", "sigmoid", "hard_sigmoid", "exponential", "linear"] {type: "string", allow-input: false}
epocas = 100 #@param {type: "slider", min: 1, max: 300, step: 1, allow-input: true}
otimizador = "sgd" #@param ["sgd", "RMSprop", "Adagrad", "Adadelta", "Adam", "Adamax", "Nadam"] {type: "string", allow-input: false}
tam_lote = 20 #@param {type: "slider", min: 1, max: 128, step: 1, allow-input:true}
qtd_neuro_cam_oculta = 60 #@param {type: "slider", min: 1, max: 125, step: 1, allow-input:true}
porcentagem_treinamento = 70 #@param {type: "slider", min: 1, max: 100, step: 1, allow-input:true}

_dict['funcao_perda'] = funcao_perda
_dict['funcao_ativacao_cam_oculta'] = funcao_ativacao_cam_oculta
_dict['funcao_ativacao_cam_saida'] = funcao_ativacao_cam_saida
_dict['epocas'] = epocas
_dict['otimizador'] = otimizador
_dict['tam_lote'] = tam_lote
_dict['qtd_neuro_cam_oculta'] = qtd_neuro_cam_oculta
_dict['prc_train'] = porcentagem_treinamento

dados_str = "{qtd_neuro_cam_oculta}, {funcao_ativacao_cam_oculta}, {funcao_ativacao_cam_saida}, {funcao_perda}, {otimizador}, {epocas}, {tam_lote}, {prc_train}".format(**_dict)
dados = dados_str.split(", ")

## Coding

In [8]:
dataset = pd.read_csv(_dir + "/" + file_name, sep=",")

df = dataset.copy()

df['credit_history'] = df['credit_history'].map({"A30": 5, 
                                                 "A31": 4,
                                                 "A32": 3,
                                                 "A33": 2,
                                                 "A34": 1})

df['present_employment_since'] = df['present_employment_since'].map({"A75": 5, 
                                                                     "A74": 4,
                                                                     "A73": 3,
                                                                     "A72": 2,
                                                                     "A71": 1})

df['housing'] = df['housing'].map({"A152": 3,
                                   "A151": 2,
                                   "A153": 1})

df['other_debtors_guarantors'] = df['other_debtors_guarantors'].map({"A103": 3,
                                                                     "A102": 2,
                                                                     "A101": 1})

df['property'] = df['property'].map({"A121": 4, 
                                     "A122": 3, 
                                     "A123": 2, 
                                     "A124": 1})

df['job'] = df['job'].map({"A174": 4, 
                           "A173": 3, 
                           "A172": 2, 
                           "A171": 1})


#30% das instâncias são maus pagadores
#a proporção de treinamento e teste será 700:300
#ou seja, 700 instâncias para treinamento, então preciso de 35% de cada instância para treinamento

#ou seja, preciso de 10,5% de maus pagadores do total para compor o treinamento = 
#e 24,5% de bons pagadores do total = 


#1: Good
#2: Bad

df_dummies = pd.get_dummies(df)
#print(df_dummies)


cols = list(df_dummies.columns.values)
cols.pop(cols.index('final_classification'))
df_dummies = df_dummies[cols+['final_classification']]

qtd_bad = 0
qtd_good = 0
train_dataset = []
expec_ans_train = []
index_todrop = []

for i in range(len(df_dummies.credit_amount)):
    
    if (qtd_good + qtd_bad) == (len(df_dummies.credit_amount) * 0.01 * (int(dados[7]))):
        break
        
    else:
    
        if (df_dummies.final_classification[i] == 1):
            
            if (qtd_good == round((int(dados[7]) * 0.01 * 0.7 * len(df_dummies.credit_amount)), 0)):
                continue
                
            else:
                qtd_good += 1
        
        else:
            
            if (qtd_bad == round((int(dados[7]) * 0.01 * 0.3 * len(df_dummies.credit_amount)), 0)):
                continue
                
            else:
                qtd_bad += 1
        
        data = list(df_dummies.iloc[i])
        index_todrop.append(i)
        expec_ans_train.append(data.pop())
        train_dataset.append(data)


df_dummies.drop(df_dummies.index[index_todrop], inplace=True)

expec_ans_test = list(df_dummies['final_classification'])

df_dummies.drop('final_classification', axis=1, inplace=True)
n_lines = df_dummies.shape[0]
test_dataset = []
media_3_iter = []

for i in range(len(expec_ans_test)):
    test_dataset.append(list(df_dummies.iloc[i]))

for j in range(3):

    X_train = np.array(train_dataset)
    Y_train = np.array(expec_ans_train)
    X_test = np.array(test_dataset)
    Y_test = np.array(expec_ans_test)

    model = Sequential()

    model.add(Dense(len(df_dummies.columns.values), input_dim=len(df_dummies.columns.values), activation=dados[1]))
    model.add(Dense(int(dados[0]), activation=dados[1]))
    model.add(Dense(int(dados[0]), activation=dados[1]))
    model.add(Dense(1, activation=dados[2]))

    model.compile(loss=dados[3], optimizer=dados[4], metrics=['accuracy'])
        
    model.fit(X_train, Y_train, validation_split=0.25, epochs=int(dados[5]), batch_size=int(dados[6]), verbose=False)

    #answers = model.predict(X_test, batch_size=int(dados[6]))

    scores = model.evaluate(X_train, Y_train)
    media_3_iter.append(scores[1])
    print("Acurácia média do modelo na iteração {}: {:.2f}%".format(j+1, scores[1]*100))


    #print("Todo o processo demorou {} segundos".format(time.time() - start_time))
print("Média geral das 3 iterações: {:.2f}%".format(np.mean(media_3_iter)*100))

22/22 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0000e+00
Acurácia média do modelo na iteração 1: 0.00%
22/22 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0000e+00
Acurácia média do modelo na iteração 2: 0.00%
22/22 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0000e+00
Acurácia média do modelo na iteração 3: 0.00%
Média geral das 3 iterações: 0.00%
